In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
print("PyTorch version:", torch.__version__)

In [ ]:
def scaled_dot_product(q, k, v):
    # (bs, head, seq, hs // head)
    d_k = q.shape[-1]
    attn_score = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(d_k)
    # (bs, head, seq, seq)
    attn_probs = F.softmax(attn_score, dim=-1)
    attn_probs = F.dropout(attn_probs, 0.1)
    # (bs, head, seq, hs // head)
    attn = torch.matmul(attn_probs, v)
    return attn


class SelfAttention(nn.Module):
    def __init__(self, hidden_size, n_heads):
        super().__init__()
        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)
        self.n_heads = n_heads

    def permute_for_scores(self, x):
        # x: (batch_size, seq_len, hidden_size)
        new_shape = x.shape[:-1] + (self.n_heads, -1)
        x = x.view(new_shape)
        # output: (bs, head, seq, hs // head)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states):
        # hidden_states: (batch_size, seq_len, hidden_size)
        # qkv layers
        q = self.permute_for_scores(self.q_proj(hidden_states))
        k = self.permute_for_scores(self.k_proj(hidden_states))
        v = self.permute_for_scores(self.v_proj(hidden_states))
        # core attention
        output = scaled_dot_product(q, k, v)
        # output: (bs, seq, head, hs // head)
        output.permute(0, 2, 1, 3)
        output.view(output.shape[0], output.shape[1], -1)
        return output


class Projection(nn.Module):
    def __init__(self, hidden_size, p=0.1):
        super().__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(p)
        self.layer_norm = nn.LayerNorm(hidden_size)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.layer_norm(hidden_states + input_tensor)
        return hidden_states


class Attention(nn.Module):
    def __init__(self, hidden_size, n_heads):
        super().__init__()
        self.self_attn = SelfAttention(hidden_size, n_heads)
        self.proj = Projection(hidden_size)

    def forward(self, hidden_states):
        self_output = self.self_attn(hidden_states)
        attention_output = self.proj(self_output, hidden_states)
        return attention_output

In [ ]:
class MLP(nn.Module):
    def __init__(self, hidden_size, intermediate_size, hidden_act):
        super().__init__()
        self.linear1 = nn.Linear(hidden_size, intermediate_size)
        self.hidden_act = hidden_act
        self.linear2 = nn.Linear(intermediate_size, hidden_size)

    def forward(self, data):
        out = self.linear1(data)
        if self.hidden_act == "gelu":
            out = F.gelu(out)
        else:
            out = F.relu(out)
        out = self.linear2(out)
        return out

## TorchScript

* First generation of PyTorch compiler
* Can support both **training and inference**
* Out-of-the-box optimiztaion tool

PyTorch is a dynamic graph execution framework, so we need to firstly construct the computation graph in order to conduct further optimizations.

Just-in-Time (JIT) compilation.

* Tracing mode: `torch.jit.trace`
* Scripting mode: `torch.jit.script`

### Tracing Mode
PyTorch uses **dynamic graph** representation (eager mode), which means the graph is built on-the-fly.

![](https://github.com/pytorch/pytorch/raw/master/docs/source/_static/img/dynamic_graph.gif)

Runs a model with certain inputs and "traces / records" all the operations that are executed into a graph.

We use the MLP example to illustrate.

In [ ]:
device = f"cuda:{torch.cuda.device_count() - 1}"
inp = torch.rand((16, 512, 768)).to(device) # (bs, seq, hs)
mlp = MLP(768, 3072, "gelu").to(device)
traced_mlp = torch.jit.trace(mlp, (inp,))
print(traced_mlp)

The above is the **structural representation** that describes the module hierarchy. We can check the class type of the traced module.

In [ ]:
print(type(traced_mlp), isinstance(traced_mlp, nn.Module))

We can print out the **graph representation** of the traced module, the intermediate representation (IR) mostly follows LLVM's convention.

* Graph: Similar to `llvm::Function`
* Block: Only dataflow is inside a block
* Node: Instruction
    * Analogous to `mlir::Operation`
    * Can have nested blocks inside
    * e.g., `prim::GetAttr`, `prim::CallMethod`, `prim::Constant`, `aten::gelu`
* Value: Input arguments / Output results
    * The edges in the graph
    * Single-static assignment (SSA) form: Each value has precisely one defining node
    * e.g., `%x: type` (statically typed!)

You can refer to the implementation file [ir.h](https://github.com/pytorch/pytorch/blob/master/torch/csrc/jit/ir/ir.h) for more details.

In [ ]:
print(traced_mlp.graph)

We can even print out the executable Python code.

In [ ]:
print(traced_mlp.code)

### Scripting Mode

Parses the Python source code of the model, and compiles the code into a graph.
* a subset of Python grammar
* Has a Lexer and Parser that parse Python syntax directly
    * Useful to deploy to somewhere without Python environment (no need to link CPython)
    * It cannot catch up the latest Python grammar, maintainability
    * Limits the programmer

In [ ]:
scripted_mlp = torch.jit.script(mlp)
print(scripted_mlp)

Or you can use the Python decorator
```python
# decorate a class
@torch.jit.script
class MLP(nn.Module):
    ...

# decorate a function
@torch.jit.script
def foo(x):
    ...
```

In [ ]:
print(type(scripted_mlp), isinstance(scripted_mlp, nn.Module))

Control flow nodes: `prim::If` and `prim::Loop`
* Output of the if-statement serve a similar role to the $\Phi$ node in traditional SSA control-flow graphs
* Same as `mlir::affine::yield`

In [ ]:
print(scripted_mlp.graph)

In [ ]:
print(scripted_mlp.code)

### Case Study: Operator fusion

$g(f(x_1, \cdots, x_n)) = g\circ f(x_1, \cdots, x_n)$

* Reduce kernel launch overheads
* TorchScript incorporates [NVFuser](https://pytorch.org/blog/introducing-nvfuser-a-deep-learning-compiler-for-pytorch/) as the backend fusion framework
* NVFuser is by default enabled

We want to fuse the linear bias add and the GELU operation, since both are element-wise operations.

Note: Do NOT use `timeit` or `time` in the standard Python library.

In [ ]:
import torch.utils.benchmark as benchmark
print(benchmark.Timer('mlp(inp)', globals={'mlp': mlp, 'inp': inp}, label='Vanilla').timeit(1000))
torch.cuda.empty_cache()
torch.cuda.synchronize()
print(benchmark.Timer('traced_mlp(inp)', globals={'traced_mlp': traced_mlp, 'inp': inp.detach().clone()}, label='Traced').timeit(1000))
torch.cuda.empty_cache()
torch.cuda.synchronize()
print(benchmark.Timer('scripted_mlp(inp)', globals={'scripted_mlp': scripted_mlp, 'inp': inp.detach().clone()}, label='Scripted').timeit(1000))
torch.cuda.empty_cache()
torch.cuda.synchronize()

Another attempt below:

In [ ]:
class NewMLP(nn.Module):
    def __init__(self, hidden_size, intermediate_size, hidden_act):
        super().__init__()
        self.linear1 = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.bias = nn.Parameter(torch.zeros(intermediate_size))
        self.hidden_act = hidden_act
        self.linear2 = nn.Linear(intermediate_size, hidden_size)

    def forward(self, data):
        out = self.linear1(data)
        out = out + self.bias
        if self.hidden_act == "gelu":
            out = F.gelu(out)
        else:
            out = F.relu(out)
        out = self.linear2(out)
        return out

In [ ]:
mlp = NewMLP(768, 3072, "gelu").to(device)
traced_mlp = torch.jit.trace(mlp, (inp,))
scripted_mlp = torch.jit.script(mlp)
print(benchmark.Timer('mlp(inp)', globals={'mlp': mlp, 'inp': inp.detach().clone()}, label='Vanilla').timeit(1000))
torch.cuda.empty_cache()
torch.cuda.synchronize()
print(benchmark.Timer('traced_mlp(inp)', globals={'traced_mlp': traced_mlp, 'inp': inp.detach().clone()}, label='Traced').timeit(1000))
torch.cuda.empty_cache()
torch.cuda.synchronize()
print(benchmark.Timer('scripted_mlp(inp)', globals={'scripted_mlp': scripted_mlp, 'inp': inp.detach().clone()}, label='Scripted').timeit(1000))
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [ ]:
print(traced_mlp.graph)

In [ ]:
print(traced_mlp.graph_for(inp))

In [ ]:
print(scripted_mlp.graph_for(inp))

Note: Prefer scripting a whole module more than scripting a function, since scripting function only includes the forward pass.

### Limitation

> [JIT should not force users to write ugly code](https://github.com/pytorch/pytorch/issues/48108)

* Generalization problem:
    * Dynamic control flow: It is depended on the input data of the forward function
    * Capture variables as constants (e.g., Dropout)
* Only use basic syntax of Python: no/few custom structures, no builtins, no inheritance, no Union, no **kwargs, no lambda, no dynamic types, etc.

In [ ]:
def f(x):
    return torch.sqrt(x) if x.sum() > 0 else torch.square(x)
m = torch.jit.trace(f, torch.tensor(3))
print(m.code)

In [ ]:
# https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_neo/modeling_gpt_neo.py#L269-L277
def foo(hidden_state, layer_past=None, attention_mask=None):
    # do something
    # ...
    return hidden_state, layer_past, attention_mask

In [ ]:
traced_foo = torch.jit.trace(foo, (inp, None, inp))

In [ ]:
scripted_foo = torch.jit.script(foo)
print(scripted_foo.graph)

In [ ]:
scripted_foo(inp, None, inp)

Takeaway:
* Code quality is the cost of scriptability and tracability.
* No transparency on the optimizations. Compiler passes make code complicated and hard to debug.

<!-- https://github.com/NVIDIA/Megatron-LM/blob/main/megatron/model/fused_bias_gelu.py -->

## torch.fx

> James K. Reed, Zachary DeVito, Horace He, Ansley Ussery, Jason Ansel, *[Torch.fx: Practical Program Capture and Transformation for Deep Learning in Python](https://arxiv.org/abs/2112.08429)*, MLSys, 2022.

In [ ]:
from torch import fx

Design principles:
* Prefer making program capture and transformation easy for typical models at the cost of working for all possible programs. Avoid complexity to support longtail, esoteric use cases.
* Work with tools and concepts that ML practitioners are already familiar with such as Python data structures and the publicly documented operators in PyTorch. (Fully Pythonic)
* Make the process of program capture highly configurable so users can implement their own solutions for long-tail uses. Allowing users to make one-off configurations is simpler than handling the general case.

> PyTorch is primarily used as an **eager execution** framework and program capture is only used for some specific transforms; It does not need to work for an entire program.

**Symbolic tracing**: Use abstract values (Proxy) rather than example inputs

The static control flow is directly eliminated.

In [ ]:
fx_traced_mlp = fx.symbolic_trace(mlp)
print(fx_traced_mlp)

To print out the graph IR, use `.graph`

In [ ]:
print(fx_traced_mlp.graph)

`%name: [#users=x] = <node_type>[target=mod_or_func_name](args = (%x1,), kwargs = {...})`

| Node type | Description |
| :--: | :-- |
| placeholder | input |
| call_module | call a sub-`nn.Module` |
| call_function | call a Python or PyTorch internal function (e.g., `operator.xxx`, `nn.functional.xxx`) |
| call_method | call a class method |
| get_attr | get a class attribute (e.g., parameter) |
| output | return |

* No primitive operations
* `args` and `kwargs` support immediate values that are natively supported in Python
* IR is much simpler

![](fig/fx_IR_comparison.png)

### Graph Traversal

In [ ]:
for node in fx_traced_mlp.graph.nodes:
    print(node, node.op, node.target, node.args, node.kwargs)

In [ ]:
fx_traced_mlp.graph.print_tabular()

### Graph Manipulation

#### Replace a function

In [ ]:
for node in fx_traced_mlp.graph.nodes:
    if node.op == 'call_function' and node.target == F.gelu:
        node.target = F.relu
print(fx_traced_mlp.graph)

#### Replace a module

In [ ]:
for node in fx_traced_mlp.graph.nodes:
    if node.op == 'call_module' and node.target == 'linear2': # string match
        fx_traced_mlp.register_module('new_linear2', nn.Linear(3072, 3072, bias=False).to(device)) # be careful with the device
        node.target = 'new_linear2'
        break
fx_traced_mlp.delete_all_unused_submodules()
print(fx_traced_mlp.graph)
# Need to recompile after modifying the graph
fx_traced_mlp.graph.lint()
fx_traced_mlp.recompile()
print(fx_traced_mlp)

#### Insert a node

In [ ]:
for node in fx_traced_mlp.graph.nodes:
    if node.op == 'output':
        with fx_traced_mlp.graph.inserting_before(node):
            new_node = fx_traced_mlp.graph.call_function(F.relu, args=(node.args[0],))
            node.args = (new_node,)
        break
fx_traced_mlp.graph.lint()
fx_traced_mlp.recompile()
print(fx_traced_mlp)

Run code as usual

In [ ]:
fx_traced_mlp(inp)

#### Shape Propagation

In [ ]:
# https://github.com/pytorch/pytorch/blob/master/torch/fx/passes/shape_prop.py
from torch.fx.passes.shape_prop import ShapeProp

ShapeProp(fx_traced_mlp).propagate(inp)
for node in fx_traced_mlp.graph.nodes:
    print(node, node.meta['tensor_meta'].shape)

#### Visualization

Need `pydot` to be installed.

```python
from torch.fx.passes.graph_drawer import FxGraphDrawer

g = FxGraphDrawer(fx_traced_mlp, "MLP")
g.get_main_dot_graph().create_svg()
```

Check torch.fx [codebase](https://github.com/pytorch/pytorch/tree/master/torch/fx) to see more use cases. Also see fx [tutorial](https://pytorch.org/docs/stable/fx.html).

### Limitation

* Dynamic control flow

In [ ]:
def func_to_trace(x):
    if x.sum() > 0:
        return torch.relu(x)
    else:
        return torch.neg(x)

traced = torch.fx.symbolic_trace(func_to_trace)

* Non-torch functions

In [ ]:
from math import sqrt

def normalize(x):
    return x / sqrt(len(x))

# It's valid Python code
normalize(torch.rand(3, 4))

traced = fx.symbolic_trace(normalize)

In [ ]:
torch.fx.wrap('len')
torch.fx.wrap('sqrt')

traced = torch.fx.symbolic_trace(normalize)

print(traced.code)

Similarly, [`Tracer`](https://github.com/pytorch/pytorch/blob/master/torch/fx/_symbolic_trace.py#L376) can be customized to have some *leaf_modules*.